In [1]:

import cv2 as cv
import numpy as np
import pandas as pd
import os
import xml.etree.ElementTree as ET

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers import RMSprop

print("All Done!!")

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

All Done!!


# Getting the code into a panda's DataFrame

In [2]:
# Put data into pandas dataframe: 

# Columns: file_name, object_id, mask_status, location_on_img
data = {
    'filename':[],# Picture object is in
    'file_num':[],
    'object_id':[],  # Unique ID formula = file_name number + '-' + Object number
    'mask_status':[],  # with_mask, without_mask, mask_worn_incorrectly
    'location_on_img':[]  # (xmin, ymin, xmax, ymax) --> square
}


directory_path = '/kaggle/input/face-mask-detection/annotations'
xml_files = os.listdir(directory_path)

def remove_char(string: str):
    done = ""
    for char in string: 
        if char=='1' or char=='2' or char=='3' or char=='4' or char=='5' or char=='6' or char=='7' or char=='8' or char=='9' or char=='0':
            done = done + char
    return done            

def read_xml_file(xml_file):
    tree = ET.parse(os.path.join(directory_path, xml_file))
    root = tree.getroot()
    file_name = root.find("filename").text
    
    objects = root.findall('object')  # List of objects
    for i, obj in enumerate(objects): 
        #print(str(len(objects)) + " - " + str(file_name) + "-" + str(i))
        data['filename'].append(file_name)
        data['object_id'].append(remove_char(file_name) + "-" + str(i)) 
        data['file_num'].append(remove_char(file_name))
        data['mask_status'].append(obj.find('name').text)
        data['location_on_img'].append((
            int(obj.find('bndbox/xmin').text),
            int(obj.find('bndbox/ymin').text),
            int(obj.find('bndbox/xmax').text),
            int(obj.find('bndbox/ymax').text)
        ))
                    
for file in xml_files: 
    #print(file)
    read_xml_file(file)
    
df = pd.DataFrame(data)
df['file_num'] = pd.to_numeric(df['file_num'])
df = df.sort_values('file_num')
del df['file_num']
df.head(100)

    

,filename,object_id,mask_status,location_on_img
978,maksssksksss0.png,0-0,without_mask,"(79, 105, 109, 142)"
979,maksssksksss0.png,0-1,with_mask,"(185, 100, 226, 144)"
980,maksssksksss0.png,0-2,without_mask,"(325, 90, 360, 141)"
3460,maksssksksss1.png,1-5,with_mask,"(191, 67, 221, 93)"
3461,maksssksksss1.png,1-6,with_mask,"(21, 73, 44, 93)"
...,...,...,...,...
3342,maksssksksss24.png,24-0,mask_weared_incorrect,"(105, 76, 161, 116)"
2972,maksssksksss25.png,25-0,with_mask,"(122, 68, 195, 152)"
1530,maksssksksss26.png,26-0,with_mask,"(73, 87, 174, 206)"
1531,maksssksksss26.png,26-1,with_mask,"(245, 152, 327, 239)"
